In [8]:
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [9]:
# --- Beam geometry ---
L = 20.0   # length in x
H = 1.0    # height in y
nx = 40    # number of elements along length
ny = 2    # number of elements along height

ele_type = "D2_nn4_quad"  # 2D, 4-node quadrilateral (linear)
ndof = 2                  # 2 DOFs per node (x, y)

# Generate a rectangular mesh
coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)
# coords: shape (n_nodes, 2)
# connect: shape (n_nodes_per_elem, n_elems)

mesh_img_fname = "full_code_example_2_mesh.png"
pre_demo.plot_mesh_2D(str(mesh_img_fname), ele_type, coords, connect)


In [10]:
# Apply boundary conditions:
# --- Identify boundaries ---
boundary_nodes, boundary_edges = pre.identify_rect_boundaries(
    coords, connect, ele_type, x_lower=0.0, x_upper=L, y_lower=0.0, y_upper=H
)

# 1) Clamp the left edge: fix x- and y-displacements = 0
fixed_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", None, 0.0)
fixed_right = pre.assign_fixed_nodes_rect(boundary_nodes, "right", None, 0.0)
# 2) Uniform downward traction on the top edge (y=H)
# Let q be negative in the y-direction
q = -0.01 # load per unit length in x
# For a 2D plane strain problem, this is a traction (tx, ty) = (0, q)
dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, q)
# dload_info[-2, :-1] = 0.0
# Combine boundary conditions
fixed_nodes = fixed_left
fixed_nodes = np.hstack((fixed_left, fixed_right))  # only the left edge is clamped


In [11]:
# --- Material properties ---
E = 200_000
nu = 0.3
# mu = E / (2.0 * (1.0 + nu))
# kappa = E / (3.0 * (1.0 - 2.0 * nu))
mu = E / (2.0 * (1.0 + nu))
#kappa = E / (2.0 * (1.0 - nu))
kappa = E / (3.0 * (1.0 - 2.0 * nu))

material_props = np.array([mu, kappa])
print(f"Material properties: mu={mu:.3f}, kappa={kappa:.3f}")


Material properties: mu=76923.077, kappa=166666.667


In [12]:
nr_num_steps = 1

# --- Solve with your hyperelastic solver ---
displacements_all, nr_info_all = hyperelastic_solver(
    material_props,
    ele_type,
    coords.T,      # shape (2, n_nodes)
    connect.T,     # shape (n_nodes_per_elem, n_elems)
    fixed_nodes,
    dload_info,
    nr_print=True,
    nr_num_steps=nr_num_steps,
    nr_tol=1e-10,
    nr_maxit=30,
)


Step 0, load factor = 1.000
Iteration 1, Correction=1.000000e+00, Residual=1.269309e-04, tolerance=1.000000e-10
Iteration 2, Correction=9.429358e-05, Residual=5.429588e-05, tolerance=1.000000e-10
Iteration 3, Correction=7.553435e-04, Residual=8.204719e-13, tolerance=1.000000e-10
Iteration 4, Correction=4.133983e-06, Residual=4.997827e-13, tolerance=1.000000e-10
Iteration 5, Correction=3.786648e-04, Residual=4.802365e-13, tolerance=1.000000e-10
Iteration 6, Correction=9.596879e-04, Residual=4.688559e-13, tolerance=1.000000e-10
Iteration 7, Correction=1.484391e-04, Residual=4.921998e-13, tolerance=1.000000e-10
Iteration 8, Correction=9.776326e-04, Residual=5.060132e-13, tolerance=1.000000e-10
Iteration 9, Correction=2.436758e-03, Residual=4.218091e-13, tolerance=1.000000e-10
Iteration 10, Correction=1.128345e-04, Residual=4.566476e-13, tolerance=1.000000e-10
Iteration 11, Correction=3.978201e-04, Residual=4.712360e-13, tolerance=1.000000e-10
Iteration 12, Correction=3.954814e-04, Residua

In [13]:
%matplotlib inline
final_disp = displacements_all[-1]  # shape: (n_nodes*ndof,)

# --- Compute the tip displacement from the FEA result ---
# We'll pick a node near x=L, y=H/2
tip_node = None
tol = 1e-3
for i, (x, y) in enumerate(coords):
    if abs(x - L/2) < tol and abs(y - H/2) < H/(2*ny):
        tip_node = i
        break
if tip_node is None:
    raise ValueError("Could not find tip node near x=L, y=H/2.")

tip_disp_y = final_disp[ndof*tip_node + 1]  # the y-component of displacement

# --- Compare with Euler–Bernoulli formula for small deflection ---
mu = material_props[0]
# We'll guess E = 3 mu (plane stress, near incompressible) or 2 mu(1 + nu) for plane strain, etc.
# --- Analytical Solution ---
# For a cantilever beam under uniformly distributed load q:
# Euler–Bernoulli tip deflection: w(L) = q * L^4 / (8 * E * I)
# E_eff = E * (1 - nu) / ((1 + nu) * (1 - 2*nu))
E_eff = E / (1 - nu ** 2.0)
I = H ** 3 / 12.0
w_analytical = 5 * q * L ** 4 / (384.0 * E_eff * I)
# w_analytical = q * L ** 4 / (8.0 * E_eff * I)

print(f"max deflection coordinates={coords[tip_node]}")
print(f"Computed max deflection (y): {tip_disp_y:.6f}")
print(f"Analytical Euler-Bernoulli deflection: {w_analytical:.6f}")

# --- Evaluate error ---
error = abs(tip_disp_y - w_analytical)
print(f"Absolute error = {error:.6e}")
print(f"Relative error = {error/w_analytical:.6e}")

# --- Plot the mesh with the final deformed shape ---

img_name = "full_code_example_2.gif"
fname = str(img_name)
viz.make_deformation_gif(displacements_all, coords, connect, ele_type, fname)


max deflection coordinates=[10.   0.5]
Computed max deflection (y): -0.001000
Analytical Euler-Bernoulli deflection: -0.001138
Absolute error = 1.375468e-04
Relative error = -1.209203e-01
